In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-27 09:23:57


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(year) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in Years}

UTM13N = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

BridgeUnder = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeUnder'
BridgeLRS   = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeLRS'
Signals     = ReceivedDir + '\\v10\\jg_request.gdb\\SignalizedIntersections'

2018-08-27 09:23:57


In [6]:
arcpy.management.Delete(Project_GDB + '\\Int_Points')
hsmpy3.network.CON_CreateIntPoints(Project_DIR,
                                   HSMPY_PATH,
                                   Routes[2016],
                                   Project_GDB,
                                   r'C:\Users\mr068144\Downloads\NM\Bridge_Data.xlsx',
                                   r'C:\Users\mr068144\Downloads\NM\Signal_Data.xlsx',
                                   Project_GDB + '\\Int_Points',
                                   Project_DIR + '\\App_DF.xlsx',
                                   Title='')

In [3]:
print(strftime("%Y-%m-%d %H:%M:%S"))
def FindAngle(O,P):
        import math
        if P[0] == O[0]:
             if P[1] == O[1]:
                 #arcpy.AddWarning(str(O) + str(P))
                 return(0) #1
             else:
                 if P[1] > O[1]:
                     return(90)  #2
                 if P[1] < O[1]:
                     return(270) #3
        else:
            if P[1] == O[1]:
                if P[0] > O[0]:
                    return 0 #4
                else:
                    return 180 #5
            else:
                if   (P[0] - O[0]) > 0 and (P[1] - O[1]) > 0:
                    return math.degrees(math.atan((P[1] - O[1]) / (P[0] - O[0]))) #6
                elif (P[0] - O[0]) > 0 and (P[1] - O[1]) < 0:
                    return 360 - math.degrees(math.atan(-(P[1] - O[1]) / (P[0] - O[0]))) #7
                elif (P[0] - O[0]) < 0 and (P[1] - O[1]) > 0:
                    return 180 - math.degrees(math.atan(-(P[1] - O[1]) / (P[0] - O[0]))) #8
                elif (P[0] - O[0]) < 0 and (P[1] - O[1]) < 0:
                    return 180 + math.degrees(math.atan((P[1] - O[1]) / (P[0] - O[0])))
def FindClosestPoint(PolylineList,IntPoint):
            n = len(PolylineList)
            Dist0 = ((PolylineList[0    ][0] - IntPoint[0]) ** 2 + (PolylineList[0    ][1] - IntPoint[1]) ** 2) ** 0.5
            Distn = ((PolylineList[n - 1][0] - IntPoint[0]) ** 2 + (PolylineList[n - 1][1] - IntPoint[1]) ** 2) ** 0.5
            if Dist0 <= Distn:
                return [PolylineList[0  ],PolylineList[1  ]]
            else:
                return [PolylineList[n-1],PolylineList[n-2]]
def PopCloseDeg(DegList,Eps):
    out = []
    for i,deg in enumerate(DegList):
        flag = False
        for odeg in out:
            if abs(deg-odeg)<=Eps or abs(deg-odeg)>=(360-Eps):
                flag = True
        if not flag:
            out.append(deg)
    return(out)
def OppositeDirID(InpID):
    out = InpID[:-2]
    ext = InpID[-1]
    if ext == 'P':
        return('{}-M'.format(out))
    if ext == 'M':
        return('{}-P'.format(out))
def FindDirection(O,P):
    direction = 'in' # approaching intersection w increasing milepost
    if P[2]-O[2]>0:
        direction = 'out'
    return(direction)

2018-08-22 10:55:31


In [8]:
# Reading Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Inp = Routes[2016]
Tolerance = "5 Feet"

Routes_DF = hsmpy3.common.FCtoDF(Inp,True)

F2L = hsmpy3.common.CreateOutPath(Project_GDB + '\\Routes_2016','F2L','')
arcpy.management.FeatureToLine(in_features=Inp,out_feature_class=F2L,attributes="ATTRIBUTES",cluster_tolerance=Tolerance)

print(strftime("%Y-%m-%d %H:%M:%S"))
F2L_DF = hsmpy3.common.FCtoDF(F2L,True)

2018-08-10 10:13:55
2018-08-10 10:19:51


In [12]:
print(Routes_DF.shape)
print(F2L_DF.shape)
print(list(F2L_DF))

(212703, 6)
(257652, 7)
['FID_Routes_2016', 'ROUTE_ID', 'BEG_POINT', 'END_POINT', 'partCount', 'Shape_Length', 'Shape']


In [13]:
print(strftime("%Y-%m-%d %H:%M:%S"))
Att_DF = hsmpy3.common.FCtoDF(AttTabs[2016])
Att_DF = Att_DF.sort_values(by=['ROUTE_ID','BEG_POINT'])
Att_DF['Len'] = Att_DF.END_POINT - Att_DF.BEG_POINT
Att_DF = Att_DF[(Att_DF.Len>0)]
Att_DF['SurfWid'] = 24.0
for i,r in Att_DF.iterrows():
    nl = 2
    try:
        nl = int(r.THROUGH_LANES)
    except:
        pass
    lw = 12.0
    try:
        lw = float(r.LANE_WIDTH)
    except:
        pass
    mdw = {1:0,2:4,3:4,4:6,5:6,6:6,7:6,0:0}
    md = 1
    try:
        md = int(r.MEDIAN_TYPE)
    except:
        pass
    surfwid = max(24,nl*lw+mdw[md])
    Att_DF.set_value(i,'SurfWid',surfwid)
    
    aadt = r.AADT
    try:
        aadt = int(aadt)
    except:
        aadt = 0
    Att_DF.set_value(i,'AADT',aadt)    
#Att_DF[(Att_DF.ROUTE_ID=='BL-34-P')]
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-10 10:59:10
2018-08-10 11:03:52


In [14]:
# Creating Signal Event Table
print(strftime("%Y-%m-%d %H:%M:%S"))
j = 0
SignalDF = hsmpy3.common.FCtoDF(Signals,False)
SignalMP_DF = pd.DataFrame(columns=['RID','MP','Offset'])
for i,r in SignalDF.iterrows():
    rdf = Routes_DF[(Routes_DF.ROUTE_ID == r.Route) | (Routes_DF.ROUTE_ID == OppositeDirID(r.Route))]
    for n,m in rdf.iterrows():
        j += 1
        pl = m.Shape.projectAs(NAD83_NM)
        p = arcpy.PointGeometry(arcpy.Point(X=r.Begin_Long,Y=r.Begin_Lati),hsmpy3.common.WGS1984).projectAs(NAD83_NM)
        offset = pl.distanceTo(p)
        MP   = pl.snapToLine(p).firstPoint.M
        SignalMP_DF.loc[j] = [m.ROUTE_ID,MP,offset]
display(SignalMP_DF[SignalMP_DF.Offset>150])
SignalMP_DF = SignalMP_DF[SignalMP_DF.Offset<=150]
SignalMP_DF = SignalMP_DF.sort_values(['RID','MP'])
SignalMP_DF.to_excel('Signal_Data.xlsx')
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-10 11:05:26


,RID,MP,Offset
34,FL-4030-M,7.020000,8887.388722
40,FL-4023-M,1.052000,5538.764166
62,FL-5081-M,4.060000,1819.913443
73,FL-4532-M,2.100000,13503.039079
85,FL-5004-M,2.484000,6532.801858
89,FL-5035-M,1.238000,1980.284267
92,FL-4053-M,4.911000,1607.138119
98,FL-4043-M,0.675000,3572.301862
108,FL-4560-M,1.128000,2486.494513
111,FL-4030-M,2.291000,15061.701878


2018-08-10 11:07:50


In [15]:
# Spatial Join on Bridge Data
print(strftime("%Y-%m-%d %H:%M:%S"))
BridgeSPJ = hsmpy3.common.CreateOutPath(Project_GDB + '\\Bridge','Underpass','')
arcpy.SpatialJoin_analysis(
        target_features = BridgeLRS, 
        join_features = BridgeUnder, 
        out_feature_class = BridgeSPJ, 
        join_operation = "JOIN_ONE_TO_ONE", 
        join_type = "KEEP_ALL", 
        match_option = "CLOSEST", 
        search_radius = '50 Feet', 
        distance_field_name = "")
FL = ['Route_ID','Begin_Poin','End_Point','UNDER_ST']
arcpy.management.DeleteField(BridgeSPJ,[f.name for f in arcpy.ListFields(BridgeSPJ) if not f.name in FL and not f.required])
Bridge_DF = hsmpy3.common.AttributeTabletoDF(BridgeSPJ,True)

print(strftime("%Y-%m-%d %H:%M:%S"))
BridgeMP_DF = pd.DataFrame(columns=['RID','BMP','EMP','UNDER_ST','Offset'])
j = 0
spat_ref = hsmpy3.common.GetSpatialReference(BridgeSPJ)
for i,r in Bridge_DF.iterrows():
    j += 1
    BridgeMP_DF.loc[j] = [r.Route_ID,r.Begin_Poin,r.End_Point,r.UNDER_ST,0]
    rdf = Routes_DF[Routes_DF.ROUTE_ID == OppositeDirID(r.Route_ID)]
    for n,m in rdf.iterrows():
        j += 1
        pl = m.Shape.projectAs(NAD83_NM)
        p1 = arcpy.PointGeometry(arcpy.Point(X=r.Shape.firstPoint.X,Y=r.Shape.firstPoint.Y),spat_ref).projectAs(NAD83_NM)
        offset1 = pl.distanceTo(p1)
        BMP   = pl.snapToLine(p1).firstPoint.M
        p2 = arcpy.PointGeometry(arcpy.Point(X=r.Shape.lastPoint.X,Y=r.Shape.lastPoint.Y),spat_ref).projectAs(NAD83_NM)
        offset2 = pl.distanceTo(p2)
        EMP   = pl.snapToLine(p2).firstPoint.M
        BridgeMP_DF.loc[j] = [m.ROUTE_ID,BMP,EMP,r.UNDER_ST,np.mean([offset1,offset2])]
        #print(m.ROUTE_ID,BMP,EMP,r.UNDER_ST,offset1,offset2)
BridgeMP_DF['Length'] = [(r['EMP']-r['BMP'])*5280 for i,r in BridgeMP_DF.iterrows()]
display(BridgeMP_DF[(BridgeMP_DF.Offset>1000) | (BridgeMP_DF.Length<=0)])
BridgeMP_DF = BridgeMP_DF[(BridgeMP_DF.Offset<=1000) & (BridgeMP_DF.Length>0)]
BridgeMP_DF = BridgeMP_DF.sort_values(['RID','BMP'])
BridgeMP_DF.to_excel('Bridge_Data.xlsx')
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-10 11:07:50
2018-08-10 11:08:26
26913


,RID,BMP,EMP,UNDER_ST,Offset,Length
5,NM-90-M,31.4160,31.4160,None,39630,0.0
8,US-380-M,153.6720,153.6720,None,87165.1,0.0
17,NM-6-M,31.7510,31.7510,None,130313,0.0
19,NM-118-M,16.4440,16.4440,None,70211.3,0.0
25,NM-4-M,63.9880,63.9880,None,5194.28,0.0
27,US-180-M,163.2550,163.2550,None,76329.7,0.0
31,US-380-M,153.6720,153.6720,None,37078.1,0.0
35,US-82-M,107.4610,107.4610,None,18762.2,0.0
40,US-285-M,297.0860,297.0860,None,241278,0.0
44,US-84-M,161.8060,161.8060,None,255917,0.0


2018-08-10 11:10:39


In [16]:
# Creating Intersection Event Table
print(strftime("%Y-%m-%d %H:%M:%S"))
#Inp = Project_GDB + '\\Sample_Routes'
#Tolerance = "5 Feet"
#F2L = Project_GDB + '\\Routes_2016_F2L'
#F2L_DF = hsmpy3.common.AttributeTabletoDF(F2L,True)

print(strftime("%Y-%m-%d %H:%M:%S"))
Int_M = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','Int_M','')
arcpy.analysis.Intersect(
    in_features=Inp,
    out_feature_class=Int_M,
    join_attributes='ALL',
    output_type='POINT',
    cluster_tolerance=Tolerance
)
Int = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','Int','')
arcpy.management.MultipartToSinglepart(in_features=Int_M,out_feature_class=Int)
#arcpy.management.AddXY(Int)
arcpy.DeleteIdentical_management(in_dataset=Int,fields='Shape')
arcpy.management.DeleteField(Int,[f.name for f in arcpy.ListFields(Int) if not f.required])

arcpy.management.AddXY(Int)
F_2_Keep = ['POINT_M', 'POINT_X', 'POINT_Y','Shape']

#Spatial join with signals and underpasses
IntSPJ1 = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','IntSPJ1','')
arcpy.SpatialJoin_analysis(
        target_features = Int, 
        join_features = BridgeSPJ, 
        out_feature_class = IntSPJ1, 
        join_operation = "JOIN_ONE_TO_ONE", 
        join_type = "KEEP_ALL", 
        match_option = "CLOSEST", 
        search_radius = '150 Feet', 
        distance_field_name = "Dist_Bridge")
arcpy.management.AlterField(IntSPJ1,'Route_ID','Route_Bridge')
F_2_Keep.extend(['Route_Bridge','Begin_Poin','End_Point','UNDER_ST'])
try:
    arcpy.management.DeleteField(in_table=IntSPJ1,drop_field=';'.join([f.name for f in arcpy.ListFields(IntSPJ1) if not f.required and not f.name in F_2_Keep]))
except:pass

IntSPJ2 = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','IntSPJ2','')
arcpy.SpatialJoin_analysis(
        target_features = IntSPJ1, 
        join_features = Signals, 
        out_feature_class = IntSPJ2, 
        join_operation = "JOIN_ONE_TO_ONE", 
        join_type = "KEEP_ALL", 
        match_option = "CLOSEST", 
        search_radius = '250 Feet', 
        distance_field_name = "Dist_Signal")
arcpy.management.AlterField(IntSPJ2,'Route','Route_Signal')
F_2_Keep.extend(['Route_Signal'])
try:
    arcpy.management.DeleteField(in_table=IntSPJ2,drop_field=';'.join([f.name for f in arcpy.ListFields(IntSPJ2) if not f.required and not f.name in F_2_Keep]))
except:pass

print(strftime("%Y-%m-%d %H:%M:%S"))
EventTable = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','EveTab','')
arcpy.LocateFeaturesAlongRoutes_lr(
        in_features                = IntSPJ2, 
        in_routes                  = Inp, 
        route_id_field             = RouteID, 
        radius_or_tolerance        = Tolerance, 
        out_table                  = EventTable, 
        out_event_properties       = " ".join(['RID', "POINT", "MP"]),
        route_locations            = "ALL", 
        in_fields                  = "FIELDS", 
        m_direction_offsetting     = "M_DIRECTON"
)

print(strftime("%Y-%m-%d %H:%M:%S"))
IntAppSPJ = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','IntAppSPJ','')
arcpy.SpatialJoin_analysis(
        target_features = IntSPJ2, 
        join_features = F2L, 
        out_feature_class = IntAppSPJ, 
        join_operation = "JOIN_ONE_TO_MANY", 
        join_type = "KEEP_ALL", 
        match_option = "INTERSECT", 
        search_radius = Tolerance, 
        distance_field_name = "Dist_F2L")

print(strftime("%Y-%m-%d %H:%M:%S"))
#display(F2L_SPJ_DF)

2018-08-10 11:10:39
2018-08-10 11:10:39
2018-08-10 11:28:51
2018-08-10 13:11:45
2018-08-10 13:13:39


In [17]:
# Create Intersections Data Frame
print(strftime("%Y-%m-%d %H:%M:%S"))
df = hsmpy3.common.AttributeTabletoDF(FC=Project_GDB + '\\CreateInt_Int',readGeometry=True)
df['X'] = [s.firstPoint.X for s in df.Shape]
df['Y'] = [s.firstPoint.Y for s in df.Shape]
df = df[['X','Y']]
df.index = [i+1 for i in range(len(df))]
ID = 1
df['Int_ID'] = [0 for i in range(len(df))]
X = df.loc[ID]['X']
Y = df.loc[ID]['Y']
df.set_value(1,'Int_ID',1)
f = FloatProgress(min=0, max=len(df))
display(f)

for i in range(len(df)+1)[2:]:
    if abs(df.loc[i]['X']-X)<0.1 and abs(df.loc[i]['Y']-Y)<0.1:
        df.set_value(i,'Int_ID',ID)
    else:
        ID += 1
        df.set_value(i,'Int_ID',ID)
        X = df.loc[i]['X']
        Y = df.loc[i]['Y']
    f.value += 1
df.index = df.Int_ID
df = df.sort_index()
Int_DF = df.copy()
Int_DF.to_excel('Int_DF.xlsx')
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-10 13:13:39


FloatProgress(value=0.0, max=155798.0)

2018-08-10 13:18:49


In [30]:
# Create Intersections App Data Frame
print(strftime("%Y-%m-%d %H:%M:%S"))
Bridge_Tolerance = 50
Signal_Tolerance = 150

IntAppSPJ = Project_GDB + '\\CreateInt_IntAppSPJ'
F2L_SPJ_DF = hsmpy3.common.FCtoDF(IntAppSPJ,False)

f = FloatProgress(min=0, max=Int_DF.shape[0])
display(f)

App_DF = pd.DataFrame(columns=['Int_ID','RID','MP','Direction','Deg','Shape','OnBridge','HasSignal'])
z = 0
for i,r in Int_DF.iterrows():
    x = r.X
    y = r.Y
    spj_df = F2L_SPJ_DF[(F2L_SPJ_DF.POINT_X==x) & (F2L_SPJ_DF.POINT_Y==y)]
    jfid = spj_df.JOIN_FID
    f2l_df = F2L_DF[F2L_DF.OBJECTID.isin(list(spj_df.JOIN_FID))]
    for j,k in f2l_df.iterrows():
        z += 1
        RID = k.ROUTE_ID
        
        P1,P2 = FindClosestPoint([[p.X,p.Y,p.M] for p in list(k.Shape)[0]],[x,y])
        MP = P1[2]
        
        Direction = FindDirection(P1,P2)
        
        Deg = FindAngle(P1,P2)
        
        onBridge = False
        bdf = BridgeMP_DF[(BridgeMP_DF.RID==RID) & 
                        ((BridgeMP_DF.BMP-Bridge_Tolerance/5280.0)<=MP) & 
                        ((BridgeMP_DF.EMP+Bridge_Tolerance/5280.0)>=MP)]
        if len(bdf)>0:
            onBridge = True
        
        HasSignal = False
        sdf = SignalMP_DF[(SignalMP_DF.RID==RID) & 
                          ((SignalMP_DF.MP-MP<=Signal_Tolerance/5280.0) &
                           (SignalMP_DF.MP-MP>=(-Signal_Tolerance/5280.0)))]
        if len(sdf)>0:
            HasSignal = True

        App_DF.loc[z] = [i,RID,MP,Direction,Deg,k.Shape,onBridge,HasSignal]
    f.value += 1
App_DF = App_DF.sort_values(by=['RID','MP','Direction'])
App_DF['Length'] = [r.Shape.length for i,r in App_DF.iterrows()]
App_DF.to_excel('App_DF.xlsx')

#display(Int_DF)
#display(App_DF)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-13 07:34:23


KeyboardInterrupt: 

In [29]:
# Filter for Intersections - removing interchanges, on/off ramp
# Filter for all ramps
# Filter for forks
print(strftime("%Y-%m-%d %H:%M:%S"))
f = FloatProgress(min=0, max=Int_DF.shape[0])
display(f)
Int_DF['IsInt'] = False
Int_DF['App1'] = ''
Int_DF['App2'] = ''
Int_DF['Degs'] = ''

for i,r in Int_DF.iterrows():
    app_df = App_DF[(App_DF.Int_ID==i) & (App_DF.OnBridge==False)]
    if(len(app_df))>0:
        app_df = app_df[[(m.RID.split('-')[0] != 'I') for n,m in app_df.iterrows()]]
        all_app = list(set(['-'.join(s.split('-')[0:2]) for s in app_df.RID]))
        all_app_i = list(set([s.split('-')[0] for s in app_df.RID]))
        Int_DF.set_value(i,'App1',all_app)
        Int_DF.set_value(i,'App2',all_app_i)
        DegList = PopCloseDeg(list(app_df.Deg),8)
        Int_DF.set_value(i,'Degs',DegList)
        if len(DegList)>=3 and len(all_app)>1 and all_app_i != ['IX']:
            Int_DF.set_value(i,'IsInt',True)
    f.value += 1

display(Int_DF[Int_DF.IsInt==False])
Int_DF.to_csv('Int_DF{}.csv'.format(strftime("%Y%m%d_%H%M%S")))
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-13 07:33:32


FloatProgress(value=0.0, max=155798.0)

KeyboardInterrupt: 

In [ ]:
# Create Intx Clusters 1
print(strftime("%Y-%m-%d %H:%M:%S"))
f = FloatProgress(min=0, max=Int_DF.shape[0])
display(f)
Cluster_Tolerance = 30  # Meters
Cluster_Dict = []
for i,r in Int_DF.iterrows():
    if r.IsInt == True:
        adf = App_DF[(App_DF.Int_ID==i) & (App_DF.Length<=Cluster_Tolerance)]
        l = [i]
        for n,m in adf.iterrows():
            g = list(App_DF[(App_DF.RID==m.RID) & (App_DF.Length==m.Length) & (App_DF.Int_ID!=m.Int_ID)]['Int_ID'])
           #g = [k for k in g if Int_DF.loc[k]['IsInt']==True]
            l.extend(g)
        Cluster_Dict.append(l)
    f.value += 1
    
print(strftime("%Y-%m-%d %H:%M:%S"))
df = pd.DataFrame(columns=range(0,6))
for c in df.columns:
    df[c]=[0 for i in range(len(Cluster_Dict))]
df
for i,l in enumerate(Cluster_Dict):
    l = list(set(l))
    l.sort()
    for j,k in enumerate(l):
        df.set_value(i,j,k)
df = df.sort_values([0,1,2,3,4,5])
df = df.drop_duplicates(subset=[0,1,2,3,4,5])
df['Len']  = [len([r[c] for c in df.columns if r[c]>0]) for i,r in df.iterrows()]
df = df.sort_values(['Len',0,1,2,3,4,5])
df1 = df[df.Len==1]
df2 = df[df.Len==2]
df2 = df2.sort_values([0])
df3 = df[df.Len==3]
df3 = df3.sort_values([0])
df4 = df[df.Len==4]
df4 = df4.sort_values([0])
df5 = df[df.Len==5]
df5 = df5.sort_values([0])
df6 = df[df.Len==6]
df6 = df6.sort_values([0])

df3['MergeID'] = 0
for i,r in df3.iterrows():
    rdf3 = df3[(df3[0]==r[0]) | (df3[0]==r[1]) | (df3[0]==r[2]) | 
               (df3[1]==r[0]) | (df3[1]==r[1]) | (df3[1]==r[2]) | 
               (df3[2]==r[0]) | (df3[2]==r[1]) | (df3[2]==r[2])]
    if len(rdf3)>0:
        merge = max(df3.MergeID) + 1
        for j in list(rdf3.index):
            if df3.loc[j]['MergeID'] == 0:
                df3.set_value(j,'MergeID',merge)
df3 = df3.sort_values('MergeID')
df4['MergeID'] = 0
for i,r in df4.iterrows():
    rdf4 = df4[(df4[0]==r[0]) | (df4[0]==r[1]) | (df4[0]==r[2]) | (df4[0]==r[3]) |
               (df4[1]==r[0]) | (df4[1]==r[1]) | (df4[1]==r[2]) | (df4[1]==r[3]) |
               (df4[2]==r[0]) | (df4[2]==r[1]) | (df4[2]==r[2]) | (df4[2]==r[3]) |
               (df4[3]==r[0]) | (df4[3]==r[1]) | (df4[3]==r[2]) | (df4[3]==r[3])]
    if len(rdf4)>0:
        merge = max(df4.MergeID) + 1
        for j in list(rdf4.index):
            if df4.loc[j]['MergeID'] == 0:
                df4.set_value(j,'MergeID',merge)
df4 = df4.sort_values('MergeID') 

df5['MergeID'] = 0
for i,r in df5.iterrows():
    rdf5 = df5[(df5[0]==r[0]) | (df5[0]==r[1]) | (df5[0]==r[2]) | (df5[0]==r[3]) | (df5[0]==r[4])]
    if len(rdf5)>0:
        merge = max(df5.MergeID) + 1
        for j in list(rdf5.index):
            if df5.loc[j]['MergeID'] == 0:
                df5.set_value(j,'MergeID',merge)
df5 = df5.sort_values('MergeID') 

df6['MergeID']  = 1

In [63]:
print(strftime("%Y-%m-%d %H:%M:%S"))
l1 = [[i] for i in df1[0]]
l2 = [[i,j] for i,j in zip(df2[0],df2[1])]
l3 = []
for i in list(set(df3.MergeID)):
    rdf3 = df3[df3.MergeID==i]
    l = []
    for j,r in rdf3.iterrows():
        l.extend([r[0],r[1],r[2]])
    l = list(set(l))
    l.sort()
    l3.append(l)
l4 = []
for i in list(set(df4.MergeID)):
    rdf4 = df4[df4.MergeID==i]
    l = []
    for j,r in rdf4.iterrows():
        l.extend([r[0],r[1],r[2],r[3]])
    l = list(set(l))
    l.sort()
    l4.append(l)
l5 = []
for i in list(set(df5.MergeID)):
    rdf5 = df5[df5.MergeID==i]
    l = []
    for j,r in rdf5.iterrows():
        l.extend([r[0],r[1],r[2],r[3],r[4]])
    l = list(set(l))
    l.sort()
    l5.append(l)
l6 = []
for i in list(set(df6.MergeID)):
    rdf6 = df6[df6.MergeID==i]
    l = []
    for j,r in rdf6.iterrows():
        l.extend([r[0],r[1],r[2],r[3],r[4],r[5]])
    l = list(set(l))
    l.sort()
    l6.append(l)
#l1.extend(l2)
l2.extend(l3)
l2.extend(l4)
l2.extend(l5)
l2.extend(l6)
#Cluster_List = l1
print(len(l2))
print(strftime("%Y-%m-%d %H:%M:%S"))


3211


In [64]:
print(strftime("%Y-%m-%d %H:%M:%S"))
Cluster_Dict = l2
FLPrg = FloatProgress(min=0, max=len(Cluster_Dict))
display(FLPrg)
fl = []
s = []
for i in range(len(Cluster_Dict)):
    f = [k for k in Cluster_Dict[i]]
    for j in range(i+1,len(Cluster_Dict)):
        d = [k for k in Cluster_Dict[j] if not k in s]
        if len(d)>0:
            if True in [(k in d) for k in Cluster_Dict[i]]:
                f.extend(Cluster_Dict[j])
                f = list(set(f))
    if not True in [(k in s) for k in f]:
            #print(f)
            fl.append(f)
            s.extend(f)
    FLPrg.value += 1
Cluster_List = fl
Cluster_List.extend(l1)
print(strftime("%Y-%m-%d %H:%M:%S"))

FloatProgress(value=0.0, max=3211.0)

In [ ]:
print(strftime("%Y-%m-%d %H:%M:%S"))
Int_DF['ClusterNum'] = 0
cnum = 0
for i in Cluster_List:
    cnum += 1
    for j in i:
        Int_DF.set_value(j,'ClusterNum',cnum)
Int_DF = Int_DF.sort_values('ClusterNum')
Int_DF.to_csv('Int_DF{}.csv'.format(strftime("%Y%m%d_%H%M%S")))

Int_DF2 = pd.DataFrame(columns=['Int_ID2','X','Y','Int_ID_List'])
cnum = 0
for i in Cluster_List:
    cnum += 1
    mpnt = arcpy.Multipoint(arcpy.Array([arcpy.Point(X=Int_DF.loc[j]['X'],Y=Int_DF.loc[j]['Y']) for j in i if Int_DF.loc[j]['IsInt']==True]))
    Int_DF2.set_value(cnum,'Int_ID2',cnum)
    Int_DF2.set_value(cnum,'X',mpnt.centroid.X)
    Int_DF2.set_value(cnum,'Y',mpnt.centroid.Y)
    Int_DF2.set_value(cnum,'Int_ID_List',i)
Int_DF2
Int_DF2.to_csv('Int_DF2_{}.csv'.format(strftime("%Y%m%d_%H%M%S")))
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
# Create Final Filtered Intx DF with Traff Control and Legs
Cluster_Tolerance = 30  # Meters
print(strftime("%Y-%m-%d %H:%M:%S"))
f = FloatProgress(min=0, max=Int_DF2.shape[0])
display(f)
Int_DF2['Legs'] = 0
Int_DF2['Signal'] = False
App_DF['Int_ID2'] = 0
for i,r in Int_DF2.iterrows():
    exclude = []
    app_df = App_DF[App_DF.Int_ID.isin(r['Int_ID_List']) & (App_DF.Length>=Cluster_Tolerance)]
    if True in list(App_DF[App_DF.Int_ID.isin(r['Int_ID_List'])]['HasSignal']):
        Int_DF2.set_value(i,'Signal',True)
    for n,m in app_df.iterrows():
        if OppositeDirID(m.RID) in list(app_df.RID):
            a = m.RID.split('-')[:-1]
            a.append('M')
            exclude.append('-'.join(a))
    exclude = list(set(exclude))
    if len(exclude)>0:
        app_df = app_df[~(app_df.RID.isin(exclude))]
    #display(app_df)
    Int_DF2.set_value(i,'Legs',len(PopCloseDeg(list(app_df.Deg),8)))
    #print(legs)
    for j in app_df.index:
        App_DF.set_value(j,'Int_ID2',i)
    f.value += 1
Int_DF3 = Int_DF2[Int_DF2.Legs>2]
Int_DF3.to_csv('Int_DF3_{}.csv'.format(strftime("%Y%m%d_%H%M%S")))
App_DF.to_csv('App_DF_{}.csv'.format(strftime("%Y%m%d_%H%M%S")))
display(Int_DF2[Int_DF2.Legs<=2])
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
# Create Intx App Event Table
print(strftime("%Y-%m-%d %H:%M:%S"))
f = FloatProgress(min=0, max=Routes_DF.shape[0])
display(f)
App_Tab = pd.DataFrame(columns=['RID','BMP','EMP','IntProx','Int_ID2','Major'])
x = 0
for i,r in Routes_DF.iterrows():
    rid = r.ROUTE_ID
    app_all = App_DF[(App_DF.RID==rid) & (App_DF.Int_ID2!=0) & (~pd.isnull(App_DF.MP))]
    mp_List = list(set(app_all.MP))
    mp_List.sort()
    int_id_List = [list(app_all[app_all.MP==mp]['Int_ID2'])[0] for mp in mp_List]
    if False in [(int_id in Int_DF3.Int_ID2) for int_id in int_id_List]:
        continue
    for int_id,mp in zip(int_id_List,mp_List):
        Int    = Int_DF3[Int_DF3.Int_ID2==int_id]
        app    = App_DF[(App_DF.RID==rid) & (App_DF.Int_ID2==int_id)]
        att    = Att_DF[(Att_DF.ROUTE_ID==rid) & (Att_DF.BEG_POINT<=mp) & (Att_DF.END_POINT>=mp)]
        app_x  = App_DF[(App_DF.Int_ID2==int_id) & (App_DF.RID!=rid)]
        att_x = pd.DataFrame()
        for n,m in app_x.iterrows():
            x_rid = m.RID 
            x_mp  = m.MP
            att_x = pd.concat([att_x,Att_DF[(Att_DF.ROUTE_ID==x_rid) & (Att_DF.BEG_POINT<=x_mp) & (Att_DF.END_POINT>=x_mp)]])
        
        int_w = [24.0]
        int_w.extend(list(att_x.SurfWid))
        int_w = float(max(int_w))/5280.0
                
        int_b = 250.0/5280.0
        Major = False
        
        AADT = [0]
        AADT.extend(list(att.AADT))
        AADT = max(AADT)

        AADT_x = [0]
        AADT_x.extend(list(att_x.AADT))
        AADT_x = max(AADT_x)
        
        if AADT>AADT_x:
            Major = True
            
        if Int.Signal.item() and Major:
            int_b = 400.0/5280.0 + int_w
        if Int.Signal.item() and not Major:
            int_b = 250.0/5280.0 + int_w
        if not Int.Signal.item() and Major:
            int_b = 150.0/5280.0 + int_w
        if not Int.Signal.item() and not Major:
            int_b = 90.0/5280.0 + int_w

        bmp = max(mp - int_w - int_b,Routes_DF.loc[i]['Shape'].firstPoint.M)
        emp = max(mp - int_w,Routes_DF.loc[i]['Shape'].firstPoint.M)
        if emp>bmp:
            x += 1
            App_Tab.loc[x] = [rid,bmp,emp,1,int_id,{True:1,False:0}[Major]]

        bmp = max(mp - int_w,Routes_DF.loc[i]['Shape'].firstPoint.M)
        emp = min(mp + int_w,Routes_DF.loc[i]['Shape'].lastPoint.M)
        if emp>bmp:
            x += 1
            App_Tab.loc[x] = [rid,bmp,emp,2,int_id,{True:1,False:0}[Major]]
        
        bmp = min(mp + int_w,Routes_DF.loc[i]['Shape'].lastPoint.M)
        emp = min(mp + int_w + int_b,Routes_DF.loc[i]['Shape'].lastPoint.M)
        if emp>bmp:
            x += 1
            App_Tab.loc[x] = [rid,bmp,emp,1,int_id,{True:1,False:0}[Major]]
    f.value += 1
App_Tab.to_csv('App_Tab{}.csv'.format(strftime("%Y%m%d_%H%M%S")))
App_Tab

In [44]:
# Convert DF to Int FC
DF2FC = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','Final','')
arcpy.management.CreateFeatureclass(out_name=os.path.basename(DF2FC),
                                    out_path=Project_GDB,geometry_type='POINT',spatial_reference=NAD83_NM)
arcpy.management.AddField(DF2FC,'Int_ID','LONG')
#arcpy.management.AddField(DF2FC,'AppList','TEXT',field_length=900)
#arcpy.management.AddField(DF2FC,'BridgeList','TEXT')
#arcpy.management.AddField(DF2FC,'AppList3','TEXT')
#arcpy.management.AddField(DF2FC,'AppAngle','TEXT')
arcpy.management.AddField(DF2FC,'Signal','Short')
arcpy.management.AddField(DF2FC,'Legs'     ,'Short')
#arcpy.management.AddField(DF2FC,'Selected','Short')
f = FloatProgress(min=0, max=Int_DF3.shape[0],description='Insert Cursor')
display(f)

ic = arcpy.InsertCursor(DF2FC)
for i,r in Int_DF3.iterrows():
    if 1>0:
        row = ic.newRow()
        row.setValue('Int_ID',r.Int_ID2)
        #try:
        #    row.setValue('AppList',r.App1)
        #except:pass
        #row.setValue('BridgeList',str(r.BridgeList))
        #row.setValue('AppList3',str(r.App3))
        row.setValue('Signal',{True:1,False:0}[r.Signal])
        row.setValue('Legs',r.Legs)
        #row.setValue('AppAngle',';'.join(['{}:{:0.1f}'.format(list(d.keys())[0],d[list(d.keys())[0]]) for d in r.AppAngle]))
        #if r.IsInt:
        #    row.setValue('Selected',1)
        #else:
        #    row.setValue('Selected',0)
        p = arcpy.PointGeometry(arcpy.Point(X=r.X,Y=r.Y),UTM13N).projectAs(NAD83_NM)
        row.shape =  p
        ic.insertRow(row)
    f.value += 1
del ic
del row

FloatProgress(value=0.0, description='Insert Cursor', max=103874.0)

In [ ]:
App_Tab = hsmpy3.common.CreateOutPath(Project_GDB + '\\CreateInt','AppTab','')
arcpy.management.CreateTable(out_name=os.path.basename(App_Tab),out_path=Project_GDB)
arcpy.management.AddField(App_Tab,'RID','Text')
arcpy.management.AddField(App_Tab,'MP','Double')
arcpy.management.AddField(App_Tab,'Direction','Text')
arcpy.management.AddField(App_Tab,'Int_ID2','Long')
arcpy.management.AddField(App_Tab,'RID','Text')